<a href="https://colab.research.google.com/github/Aditi0102/ML-notebook/blob/main/AiModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install -q kaggle

In [6]:
%tensorflow_version 2.x

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import cv2
from tensorflow.keras.optimizers import RMSprop
from google.colab.patches import cv2_imshow
import math

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aditiagarwal01","key":"4c56849565c39183e765f285aeafe7d3"}\n'}

In [9]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

100% 3.58G/3.58G [01:04<00:00, 19.0MB/s]
100% 3.58G/3.58G [01:05<00:00, 59.1MB/s]


In [11]:
!unzip \*.zip  && rm *.zip
clear_output()
print("Extracted")

Extracted


In [12]:
import os
train_folder = 'Directories/training/violent'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

In [13]:
# Training Data

count = 0
for i in range (1,800):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (150,150))
        fname = '%s/%d.jpg' % (train_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [14]:
import os
test_folder = 'Directories/validation/violent'
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

In [15]:
# Testing Data
count = 0
for i in range (800,1000):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (150,150))
        fname = '%s/%d.jpg' % (test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [16]:
import os
nv_folder = 'Directories/training/non_violent'
if not os.path.exists(nv_folder):
    os.makedirs(nv_folder)

In [17]:
count = 0
for i in range (1,800):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (150,150))
        fname = '%s/%d.jpg' % (nv_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")

Done Training with Non-Violence !!


In [18]:
import os
nv_test_folder = 'Directories/validation/non_violent'
if not os.path.exists(nv_test_folder):
    os.makedirs(nv_test_folder)

In [19]:
count = 0
for i in range (800,1000):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (150,150))
        fname = '%s/%d.jpg' % (nv_test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")


Done Training with Non-Violence !!


In [20]:
base_dir = './Directories'
train_dir = os.path.join(base_dir,'training')
validation_dir = os.path.join(base_dir,'validation')

In [21]:
train_violent_dir = os.path.join(train_dir,'violent')
train_nonviolent_dir=os.path.join(train_dir,'non_violent')
validation_violent_dir=os.path.join(validation_dir,'violent')
validation_nonviolent_dir=os.path.join(validation_dir,'non_violent')

In [22]:
train_datagen= ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale =1./255)

In [23]:
train_generator = train_datagen.flow_from_directory(train_dir,color_mode="rgb", target_size = (150,150),batch_size=20, class_mode='binary', shuffle=True)

Found 226270 images belonging to 2 classes.


In [24]:
validation_generator = test_datagen.flow_from_directory(validation_dir,color_mode="rgb",target_size=(150,150),batch_size=20,class_mode='binary', shuffle=True)


Found 54195 images belonging to 2 classes.


In [25]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [26]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
model1= model.fit(train_generator, 
                  steps_per_epoch=100,
                  epochs=100, 
                  validation_data=validation_generator,
                  validation_steps=50,verbose=2)

Epoch 1/100
100/100 - 22s - loss: 0.6456 - accuracy: 0.6070 - val_loss: 0.8984 - val_accuracy: 0.5240
Epoch 2/100
100/100 - 15s - loss: 0.6248 - accuracy: 0.5965 - val_loss: 0.7760 - val_accuracy: 0.4850
Epoch 3/100
100/100 - 15s - loss: 0.6088 - accuracy: 0.6430 - val_loss: 0.7950 - val_accuracy: 0.4210
Epoch 4/100
100/100 - 15s - loss: 0.6089 - accuracy: 0.6385 - val_loss: 0.7894 - val_accuracy: 0.5420
Epoch 5/100
100/100 - 15s - loss: 0.6003 - accuracy: 0.6320 - val_loss: 0.8030 - val_accuracy: 0.4840
Epoch 6/100
100/100 - 15s - loss: 0.5769 - accuracy: 0.6650 - val_loss: 0.8060 - val_accuracy: 0.5560
Epoch 7/100
100/100 - 15s - loss: 0.5804 - accuracy: 0.6625 - val_loss: 0.8247 - val_accuracy: 0.5370
Epoch 8/100
100/100 - 15s - loss: 0.5812 - accuracy: 0.6540 - val_loss: 0.7922 - val_accuracy: 0.5390
Epoch 9/100
100/100 - 15s - loss: 0.5713 - accuracy: 0.6615 - val_loss: 0.7780 - val_accuracy: 0.5300
Epoch 10/100
100/100 - 15s - loss: 0.5712 - accuracy: 0.6670 - val_loss: 0.9755 - 

In [28]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1611330564.h5
